In [ ]:
import com.github.ax_as.datapro.data.*
import java.nio.file.Path
import org.jetbrains.kotlinx.dataframe.DataFrame
import org.jetbrains.kotlinx.dataframe.api.JoinType

fun toLocalPath(file: String) = Path.of("/data/files/output/$file")
val htmlAc = toLocalPath("ranking_perderam_mais_posicoes_ac.html")
val htmlAcInju = toLocalPath("ranking_perderam_mais_posicoes_ac_inj.html")
val htmlAcSemZero =
    toLocalPath("ranking_perderam_mais_posicoes_ac_naozerados.html")

val configuration = DisplayConfiguration(
    rowsLimit = null,
    useDarkColorScheme = true,
)

var inj = DataFrame.fromLocal(DataFiles.injusticados)
var listaAc = DataFrame.concursoSerpro(DataFiles.serpro_ac).sortByDesc("nota_final")
var listaPPP = DataFrame.concursoSerpro(DataFiles.serpro_ppp).sortByDesc("nota_final")
var listaPcd = DataFrame.concursoSerpro(DataFiles.serpro_pcd).sortByDesc("")
var bb = DataFrame.concursoBB(DataFiles.bb)

var allSerpro = listaAc.concat(listaPPP, listaPcd)
    .sortByDesc { it["nota_final"] }.remove("pos").add("pos") { (index() + 1) }
    .moveToLeft("pos")

allSerpro.join(bb, JoinType.Left) {
    it["nome"] match right["nome"]
}
    .toDefaultHtml().openInBrowser()

listaAc.sortByPosDiff().toDefaultHtml().writeHTML(htmlAc)
listaAc.filter {
    "notapratica"<Double>() > 0
}.sortByPosDiff().toDefaultHtml().writeHTML(htmlAcSemZero)

listaAc.sortByPosDiff().join(inj, JoinType.Inner) {
    it["inscricao"] match right["id"]
}.toDefaultHtml().writeHTML(htmlAcInju)


